Import Dependencies

In [ ]:

import openai

## Instructor is to enable structured imports rather than we putting json schema and asking LLM to fill it.
import instructor
from qdrant_client import QdrantClient

from pydantic import BaseModel, Field

Mock Example

In [ ]:
prompt = """
You are a helpful assistant.
Return an answer to the question.
Question: What is your name? How do u differ from Gemini model?
"""

In [ ]:
openai_response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0
)

print(openai_response.choices[0].message.content)

Structured outputs

In [ ]:
## Wraps a client of LLM provider and returns its own client with the LLM object, Hereafter we can use this client instead of OpenAI client
client = instructor.from_openai(openai.OpenAI())

Add a Pythandic schema object (to replace JSON schema we did for evaluation dataset - ipynb)

In [ ]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [ ]:
## client.chat.completions.create => only gives the response. 'with_completion' will give both original response and pydantic
## schema response
response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0,
    response_model=RAGGenerationResponse
)


In [ ]:
raw_response

In [ ]:
response

In [ ]:
## This itself is a context prompt engineering. The description of the fields are injected.
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    reasoning: str = Field(description="The reasoning for the answer")

In [ ]:
response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0,
    response_model=RAGGenerationResponse
)

In [ ]:
response

RAG Example

In [ ]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


## This changes, see the instructor response model is added here (Pydantic inheritance from BaseModel)
def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationResponse
    )

    return response, raw_response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer, raw_response = generate_answer(prompt)

    final_result = {
        ## The RAGGenerationResponse data model
        "datamodel": answer,
        ## Actual answer
        "answer": answer.answer,
        "rawresponse": raw_response,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [ ]:
qdrant_client = QdrantClient(url="http://localhost:6333")


In [ ]:
output = rag_pipeline("Can I get a charging cable? Please suggest me a good one.", qdrant_client)

In [ ]:
output

In [ ]:
output["datamodel"]

In [ ]:
output["answer"]

In [ ]:
output["rawresponse"]